In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import re
import time

In [2]:
facilities = pd.read_csv("new_facilities.csv", dtype='string')
facilities

,Name,City,State,CCN
0,Atmore Community Hospital,Atmore,AL,010169
1,Baptist Medical Center East,Montgomery,AL,010149
2,Bullock County Hospital,Union Springs,AL,010110
3,Decatur Morgan Hospital-Decatur Campus,Decatur,AL,010085
4,Georgiana Medical Center,Georgiana,AL,010047
...,...,...,...,...
89,Astria Regional Medical Center,Yakima,WA,500012
90,Bluefield Regional Medical Center,Bluefield,WV,510071
91,Ohio Valley Medical Center,Wheeling,WV,510039
92,Summersville Regional Medical Center,Summersville,WV,510082


In [5]:
def url_ccn(ccn):
    return 'https://data.cms.gov/tools/medicare-inpatient-hospital-look-up-tool/provider/' + ccn +'?size=10&offset=0&ccn=' + ccn

service = Service(ChromeDriverManager().install())
service.SuppressInitialDiagnosticInformation = True

driver = webdriver.Chrome(service = service)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\email\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


In [7]:
driver.get(url_ccn('010169'))

address = driver.find_element(By.XPATH, "/html/body/div/div/div/div/div/div/div[1]/div/div/div/div/div/ul/li[1]")


In [18]:
text = address.text.split("\n")
print(text[0])
print("---")
print(re.findall("(?<= )[0-9]+$", text[1]))

401 Medical Park Drive
---
['36502']


In [20]:
data_to_append = []
length = len(facilities.loc[:,'CCN'])
for i in range(length):
    # get CCN to seach for
    CCN = facilities.loc[:,'CCN'][i]
    # load up page
    driver.get(url_ccn(CCN))
    time.sleep(1.8)
    # get address and zip code
    text = driver.find_element(By.XPATH, "/html/body/div/div/div/div/div/div/div[1]/div/div/div/div/div/ul/li[1]").text.split("\n")
    data_to_append.append([CCN, text[0], re.findall("(?<= )[0-9]+$", text[1])[0]])
    print(str(i+1) + " out of " + str(length) + " done!", end = "\r")

In [25]:
to_join = pd.DataFrame(data_to_append, columns=['CCN', 'Street_Address', "ZIP"])
to_join

,CCN,Street_Address,ZIP
0,010169,401 Medical Park Drive,36502
1,010149,400 Taylor Road,36117
2,010110,102 West Conecuh Avenue,36089
3,010085,1201 7th Street Se,35601
4,010047,515 N Miranda Avenue,36033
...,...,...,...
89,500012,110 South Ninth Ave,98902
90,510071,500 Cherry St,24701
91,510039,2000 Eoff Street,26003
92,510082,400 Fairview Heights Road,26651


In [26]:
out = pd.merge(facilities, to_join, how = "inner", on = "CCN")
out

,Name,City,State,CCN,Street_Address,ZIP
0,Atmore Community Hospital,Atmore,AL,010169,401 Medical Park Drive,36502
1,Baptist Medical Center East,Montgomery,AL,010149,400 Taylor Road,36117
2,Bullock County Hospital,Union Springs,AL,010110,102 West Conecuh Avenue,36089
3,Decatur Morgan Hospital-Decatur Campus,Decatur,AL,010085,1201 7th Street Se,35601
4,Georgiana Medical Center,Georgiana,AL,010047,515 N Miranda Avenue,36033
...,...,...,...,...,...,...
89,Astria Regional Medical Center,Yakima,WA,500012,110 South Ninth Ave,98902
90,Bluefield Regional Medical Center,Bluefield,WV,510071,500 Cherry St,24701
91,Ohio Valley Medical Center,Wheeling,WV,510039,2000 Eoff Street,26003
92,Summersville Regional Medical Center,Summersville,WV,510082,400 Fairview Heights Road,26651


In [27]:
out.to_csv("new_facilities_final.csv")